# Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## Loading data

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
y_train = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

## Lab 1: Multiple inputs model with functional API

### Data preparation

In [3]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:] #X_A: features 0-4, X_B: features 2-7
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

### Create Model

In [4]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           210         deep_input[0][0]                 
__________________________________________________________________________________________________
wide_input (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 30)           930         dense[0][0]                      
______________________________________________________________________________________________

### Compile and train model

In [6]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit((X_train_A, X_train_B), y_train, epochs=20, validation_data=((X_valid_A, X_valid_B), y_valid))

ValueError: Data cardinality is ambiguous:
  x sizes: 11610, 11610
  y sizes: 3870
Make sure all arrays contain the same number of samples.

### Test the model

In [ ]:
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
print("mse_test", mse_test)

In [ ]:
y_pred = model.predict((X_new_A, X_new_B))
print("y_pred", y_pred)

## Lab 2: Multiple outputs model with functional API

### Create model

In [ ]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

In [ ]:
model.summary()

### Compile and train model

In [ ]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))

history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20, validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

### Test the model

In [ ]:
total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])

print(total_loss, main_loss, aux_loss)

In [ ]:
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])
print(y_pred_main)
print(y_pred_aux)

## Lab 3: Saving and Restoring

### Create model

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

### Train the model

In [ ]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))3

### Test the model

In [ ]:
mse_test = model.evaluate(X_test, y_test)
print("mse_test", mse_test)

X_new = X_test[:3]
model.predict(X_new)

### Save and restore the model

* Save model

In [ ]:
model.save("my_keras_model.h5")

* Restore model

In [ ]:
restored_model = keras.models.load_model("my_keras_model_h5")

In [ ]:
mse_test = restored_model.evaluate(X_test, y_test)
print("mse_test", mse_test)
X_new = X_test[:3]
restored_model.predict(X_new)

## Lab 4: Using Callbacks during Training

### Setup checkpoint for callbacks method

In [7]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)

### Train and test the model

In [8]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb])

ValueError: Data cardinality is ambiguous:
  x sizes: 11610
  y sizes: 3870
Make sure all arrays contain the same number of samples.

In [ ]:
model.evaluate(X_valid, y_valid)

### Restore the model from a checkpoint

In [ ]:
restored_model = keras.models.load_model("my_keras_model.h5") # rollback to best model
restored_model.evaluate(X_valid, y_valid)

## Lab 5: Callbacks with early stopping

### Setup checkpoint for early_stopping

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)

### Train and test the model

In [ ]:
history = model.fit(X_train, y_train, epochs=1000, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
model.evaluate(X_valid, y_valid)

## Lab 6: Tensorboard

### Define logdir

In [ ]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")

In [ ]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

### Setup TensorBoard callbacks

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

### Train the model with callbacks

In [ ]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb, tensorboard_cb])

### TensorBoard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

# 연습문제
## Question 1
Build a model to predict house value using California housing data and functional API. (데이터로딩관련은 슬라이드3 참조. for loading data, see slide 3)


* np.random.seed(42)
* tf.random.set_seed(42)
* The model uses 3 inputs: input A (features 0-2), input B (features 3-5) and input C (features 6-7). See the figure.
* The number of neurons in all hidden layers is set to 30.
* Use loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3)
* Number of epoch = 20, report the mse loss on test set.

![Q1](https://blog.kakaocdn.net/dn/GTDT0/btq8UPigqKY/EJrk93xAwnWKkdseDTE5DK/img.png)

Test MSE 정답(Ans)은 0.38입니다.

### 1. 데이터 로드
### 1.1 보스턴 집가격 데이터 로드

In [9]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

boston = load_boston()

### 1.2 train,vaild,test 데이터 구분하시오.

In [10]:
X_train_full, X_test, y_train_full, y_test = train_test_split(boston.data, boston.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

### 1.3 StandardScale로 데이터 표준화 시키시오.

In [11]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
y_train = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### 1.4 모델에 Input으로 입력할 Feature 구분하시오.

In [12]:
X_train_A, X_train_B, X_train_C = X_train[:, 0:3], X_train[:, 3:6], X_train[:, 6:]
X_valid_A, X_valid_B, X_valid_C = X_valid[:, 0:3], X_valid[:, 3:6], X_valid[:, 6:]
X_test_A, X_test_B, X_test_C = X_test[:, 0:3], X_test[:, 3:6], X_test[:, 6:]

### 2. 모델 구축
### 2.1 함수형 API로 빌드하시오.
* 단 np.random.seed(42)와 tf.random.set_seed(42)로 고정시킵니다.

In [13]:
np.random.seed(42)
tf.random.set_seed(42)

input_A = keras.layers.Input(shape=[3], name="inputA")
input_B = keras.layers.Input(shape=[3], name="inputB")
input_C = keras.layers.Input(shape=[2], name="inputC")

hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(input_C)
hidden3 = keras.layers.Dense(30, activation="relu")(hidden2)

concat = keras.layers.concatenate([input_A, hidden1, hidden3])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.models.Model(inputs=[input_A, input_B, input_C], outputs=[output])


### 2.1.1 모델 형태 출력하시오.

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputC (InputLayer)             [(None, 2)]          0                                            
__________________________________________________________________________________________________
inputB (InputLayer)             [(None, 3)]          0                                            
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 30)           90          inputC[0][0]                     
__________________________________________________________________________________________________
inputA (InputLayer)             [(None, 3)]          0                                            
______________________________________________________________________________________________

### 2.2. 모델 훈련
### 2.2.1 모델 loss와 optimizer 설정

In [15]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
# 분류 - crossentropy / 회귀 - mse / 풀어야할 문제에 따라 loss 선택

### 2.2.2 모델 훈련

In [16]:
history = model.fit((X_train_A, X_train_B, X_train_C), y_train, epochs=20, validation_data=((X_valid_A, X_valid_B, X_valid_C), y_valid))

ValueError: Data cardinality is ambiguous:
  x sizes: 284, 284, 284
  y sizes: 95
Make sure all arrays contain the same number of samples.

### 3. 성능 평가하시오.
* MSE 사용해 0.4257 결과치 보이시오.

In [ ]:
mse_test = model.evaluate(X_test, y_test)
print("mse_test", mse_test)

## Question 2
MNIST 숫자손글씨 데이터셋을 이용하여 98%이상의 정확도를 갖는 다계층퍼셉트론(인공신경망)을 만드시오.

[Build a MLP on the MNIST dataset which achieves over 98% accuracy point on test set.]

* np.random.seed(42)
* tf.random.set_seed(42)

**< Tip >**
* Your model may contain from 2-3 dense layers.
* Use callbacks methods to save time

Test ACC 정답(Ans)은 0.98입니다.

### 1. 데이터 로드
### 1.1 MNIST 데이터 로드 및 기본적인 전처리 진행하시오.

In [17]:
import tensorflow as tf
from tensorflow import keras
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

11493376/11490434 [==============================] - 0s 0us/step


### 2. 모델 빌드

In [18]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [19]:
model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(learning_rate=2e-1), metrics=["accuracy"])

In [21]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_mnist_model.h5", save_best_only=True)

history = model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=(early_stopping_cb, checkpoint_cb))

Epoch 1/100
1719/1719 [==============================] - 1s 512us/step - loss: 0.0909 - accuracy: 0.9722 - val_loss: 0.0809 - val_accuracy: 0.9746

ImportError: `save_model` requires h5py.

### 3. 성능 평가
* Test ACC 정답(Ans)은 0.98입니다.

In [ ]:
mse_test = model.evaluate(X_test, y_test)
print("mse_test", mse_test)